In [1]:
from utils import load_model

model_id = 'gemma-3'
model, processor = load_model(model_id)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Gemma3ForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(4096, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (self_attn): SiglipSdpaAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=1

In [6]:
from datasets import load_dataset
# ds = load_dataset("cais/mmlu", "all")

ds = load_dataset("truthfulqa/truthful_qa", "generation")
questions_test = ds['validation']['question'][int(0.8*len(ds['validation'])):]
correct_answers_test = ds['validation']['correct_answers'][int(0.8*len(ds['validation'])):]

In [ ]:
from tqdm.auto import tqdm
import utils
import importlib
importlib.reload(utils)
from utils import generate_and_decode_new_tokens

initial_answer = []
final_answer = []
for question in tqdm(questions_test):
    res_1, res_2 = generate_and_decode_new_tokens(question, model, processor, model_id)
    initial_answer.append(res_1)
    final_answer.append(res_2)

# Compute Score

In [ ]:
from utils import generate_and_decode_new_tokens, to_request, create_anthropic_batch_job

print("Creating batch job for initial answers")
requests = [to_request(f'truthfulqa_{model_id}_initial_prompt{i}', q, a, p) 
                for i, (q, a, p) in enumerate(zip(questions_test, correct_answers_test, initial_answer))]

# create_anthropic_batch_job(requests, api_key=api_key)
import json
with open(f"truthfulqa-{model_id}_initial_prompt.jsonl", 'w') as f:
    for item in requests:
        json_line = json.dumps(item)
        f.write(json_line + '\n')
from openai import OpenAI
client = OpenAI(api_key="")
batch_input_file = client.files.create(
    file=open(f"truthfulqa-{model_id}_initial_prompt.jsonl", "rb"),
    purpose="batch"
)
batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": f"truthfulqa-{model_id}_initial_prompt"
    }
)

print("Creating batch job for final answers")
if 'gemma' in str(type(model)).lower():
    requests = [to_request(f'truthfulqa_{model_id}_final_prompt-{i}', q, a, p) 
                for i, (q, a, p) in enumerate(zip(questions_test, correct_answers_test, final_answer))]
else:
    requests = [to_request(f'truthfulqa_{model_id}_final_prompt-{i}', q, a, p) 
                for i, (q, a, p) in enumerate(zip(questions_test, correct_answers_test, final_answer))]
with open(f"truthfulqa-{model_id}_final_prompt.jsonl", 'w') as f:
    for item in requests:
        json_line = json.dumps(item)
        f.write(json_line + '\n')
# create_anthropic_batch_job(requests, api_key=api_key)
batch_input_file = client.files.create(
    file=open(f"truthfulqa-{model_id}_final_prompt.jsonl", "rb"),
    purpose="batch"
)
batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": f"truthfulqa-{model_id}_final_prompt"
    }
)